# Урок 2. Метрики качества классификации ч.1

В уроке мы разбирали матрицу ошибок и некоторые метрики, основанные на терминах матрицы ошибок. Здесь же посмотрим, как эти метрики применять на практике и что  по ним можно выяснить.

### 1.1
Выкачайте тренировочный датасет Titanic - train.csv - с сайта [kaggle](https://www.kaggle.com/c/titanic/data). С помощью функции pd.read_csv() загрузите данные в датафрейм. Выведите первые 20 строк и проанализируйте данные: какие колонки присутствуют (более конкретная информация по ним есть на сайте kaggle), каким образом в них обозначены данные и какие типы данных используются (используйте pandas.dtypes). 

In [36]:
import pandas as pd

In [37]:
df = pd.read_csv('train.csv')
df.head(5)
df.Age.isnull().sum()

177

### 1.2
Проверьте, имеются ли пропущенные значения в колонках, и выведите сумму всех пропущенных значений в каждой из колонок.

In [38]:
df.isnull().sum(axis =0)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### 1.3
Замените все пропущенные значения колонки Age на медианы в зависимости от пола человека: т.е. если пол человека в строке с пропущенным значением "male", заменяете пропущенное значение возраста на медиану по всем известным возрастам мужчин, и наоборот. Выведите медианы возраста в зависимости от пола. Пропущенные значения колонок Cabin и Embarked замените на U (Unknown).

Примечание: для группировки по полу можно использовать метод df.groupby(), который имеет встроенные агрегатные функции (в т.ч. для вычисления медианы).  Для установки пропущенных значений в датасете воспользуйтесь функцией df.apply().

In [39]:
import numpy as np

In [40]:
df.Cabin.replace(np.nan, 'U', inplace = True)
df.Embarked.replace(np.nan, 'U', inplace=True)

In [41]:
def chagne_age(row):
    if row.isnull():
        return True

In [44]:
female = df[df.Sex == 'female']['Age'].median()
male = df[df.Sex == 'male']['Age'].median()
print(female, male)

27.0 29.0


In [45]:
df.Age.fillna(1000, inplace=True)

In [46]:
def change(row):
    if row.Sex=='female' and row.Age == 1000.0:
        return female
    if row.Sex=='male' and row.Age == 1000.0:
        return male
    else:
        return row.Age

In [47]:
df.Age = df.apply(change, axis=1)

In [50]:
df.isnull().sum(axis=0)

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

### 1.4
Выведите возраст пассажиров с PassengerID = [6, 20]. Убедитесь, что заполнены все пропущенные значения (воспользуйтесь функцией df.isnull() ).

In [63]:
for step in range(6, 20):
    print (step, ':', int(df.Age[step]), 'лет')


6 : 54 лет
7 : 2 лет
8 : 27 лет
9 : 14 лет
10 : 4 лет
11 : 58 лет
12 : 20 лет
13 : 39 лет
14 : 14 лет
15 : 55 лет
16 : 2 лет
17 : 29 лет
18 : 31 лет
19 : 27 лет


Так как не все классификаторы могут работать с категориальными признаками, в данном случае необходимо заменить их на числовые.

Также можно было бы выделить расширенные признаки, такие как статус пассажира (Mr., Mrs., Miss., Dr., Master. и т.д.), собственный индекс билета, индекс палубы и другие, но здесь ограничимся выбором более простых признаков.

### 1.5
В колонке Sex замените значения на 0, если пол "male", и на 1, если "female". В колонке Embarked замените параметры "U", "S", "C", "Q" на 0, 1, 2, 3 соответственно. Отбросьте колонки PassengerId, Name, Ticket, Cabin. Выведите первые 20 строк получившегося набора данных.

In [68]:
import copy

In [71]:
df.Sex.replace('male', 0, inplace = True)
df.Sex.replace('female', 1, inplace = True)
df.Embarked.replace('U', 0, inplace=True)
df.Embarked.replace('S', 1, inplace = True)
df.Embarked.replace('C', 2, inplace = True)
df.Embarked.replace('Q', 3, inplace = True)
df1 = copy.copy(df)
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,1
1,1,1,1,38.0,1,0,71.2833,2
2,1,3,1,26.0,0,0,7.9250,1
3,1,1,1,35.0,1,0,53.1000,1
4,0,3,0,35.0,0,0,8.0500,1


### 1.6
Приступим к построению классификаторов. **Условимся, что, если функция или объект модели имеют параметр random_state, то устанавливаем его равным 17 в каждом из случаев.**

Разделите данные на тренировочный и тестовый датасеты, установив размер тестового как 0.25 (первая колонка Survived является целевой, поэтому необходимо сначала ее отделить от признаков).

In [92]:
from sklearn.model_selection import train_test_split 
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,1
1,1,1,1,38.0,1,0,71.2833,2
2,1,3,1,26.0,0,0,7.9250,1
3,1,1,1,35.0,1,0,53.1000,1
4,0,3,0,35.0,0,0,8.0500,1


In [95]:
df_tar = df.Survived
df_data = df.loc[:,'Pclass':'Embarked']
df_tar.head(2)

0    0
1    1
Name: Survived, dtype: int64

In [96]:
x_train, x_value, y_train, y_value = train_test_split(df_data, df_tar, test_size = 0.25, random_state = 17)

### 1.7
Обучите на полученных выборках несколько классификаторов, которые импортированы ниже.

**Примечание: в методе ближайших соседей используйте количество соседей, равное 5.**

In [98]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [103]:
from sklearn.metrics import r2_score, mean_squared_error

In [101]:
knn = KNeighborsClassifier(n_neighbors=5)
knn_model = knn.fit(x_train, y_train)
knn_pred = knn_model.predict(x_value)
gaus = GaussianNB()
gaus_model = gaus.fit(x_train, y_train)
gaus_pred = gaus_model.predict(x_value)
d_tree = DecisionTreeClassifier()
d_tree_model = d_tree.fit(x_train, y_train)
d_tree_pred = d_tree_model.predict(x_value)
log_reg = LogisticRegression()
log_reg_model = log_reg.fit(x_train, y_train)
log_reg_pred = log_reg_model.predict(x_value)

In [110]:
print(f'Knn model R2: {round(r2_score(y_value, knn_pred), 4)} and RMSE: {round(mean_squared_error(y_value, knn_pred),4)}')
print(f'Gaus model R2: {round(r2_score(y_value, gaus_pred), 4)} and RMSE: {round(mean_squared_error(y_value, gaus_pred),4)} ')
print(f'D_tree model R2: {round(r2_score(y_value, d_tree_pred), 4)} and RMSE: {round(mean_squared_error(y_value, d_tree_pred),4)}')
print(f'Log_reg model R2: {round(r2_score(y_value, log_reg_pred), 4)} and RMSE: {round(mean_squared_error(y_value, log_reg_pred),4) }')

Knn model R2: -0.2582 and RMSE: 0.3049
Gaus model R2: 0.1304 and RMSE: 0.2108 
D_tree model R2: 0.1489 and RMSE: 0.2063
Log_reg model R2: 0.1674 and RMSE: 0.2018


Очевидно, в этом случае использовать метрику accuracy, которую мы применяли до этого, не совсем правильно: она не даст верную оценку классификатору, потому как выживших явно меньше, чем погибших.

Вспомним про такие метрики как точность и полнота. Здесь они подойдут хорошо, так как у нас имеются положительный и отрицательный классы, и определить корректно один из них может оказаться более важным .

### 1.10
Для каждого классификатора выведите матрицу ошибок и самостоятельно рассчитайте метрику recall (расчеты должны присутствовать). Проверьте расчеты, воспользовавшись встроенной метрикой recall_score.

In [111]:
from sklearn.metrics import confusion_matrix, recall_score

In [118]:
confusion_matrix(y_value, knn_pred)
sc_knn = recall_score(y_value, knn_pred)
sc_gaus = recall_score(y_value, gaus_pred)
sc_tree = recall_score(y_value, d_tree_pred)
sc_log = recall_score(y_value, log_reg_pred)
print(f'knn: {round(sc_knn,4)} \ngaus: {round(sc_gaus,4)} \ntree: {round(sc_tree,4)} \nlog: {round(sc_log,4)}')

knn: 0.4891 
gaus: 0.7065 
tree: 0.7065 
log: 0.6957


In [119]:
sc_gaus>sc_tree

False

### 1.11
Выберите ту модель, на которой метрика recall давала лучший ответ, и рассчитайте для нее precision, используя встроенную функцию.

In [120]:
from sklearn.metrics import precision_score

In [121]:
precision_score(y_value, d_tree_pred)

0.7738095238095238